In [14]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()
import pandas as pd
import joblib
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pickle

from tensorflow.keras.models import Sequential, Model
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, f1_score, accuracy_score, precision_score, recall_score

from keras.layers import Input, Dense, GlobalAveragePooling2D, UpSampling2D, add, concatenate, Conv2D, Lambda, BatchNormalization, MaxPool2D, LSTM, GRU, Reshape, Flatten

In [2]:
X1 = np.array(joblib.load("Kfold/Video numpy/0.pkl"))
X2 = np.array(joblib.load("Kfold/Video numpy/1.pkl"))

In [3]:
df1 = pd.read_csv("Kfold/Dataframe/0", header = 0, index_col=0)
df2 = pd.read_csv("Kfold/Dataframe/1", header = 0, index_col=0)

In [4]:
df1

,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id,video_id,bbox_xmin,bbox_ymin,bbox_xmax,bbox_ymax,gloss
8755,25,-1,1,8,12,aslsearch,train,http://www.aslsearch.com/signs/videos/abdomen.mp4,0,341,190,46,571,400,abdomen
8760,25,-1,1,9,12,aslsearch,train,http://www.aslsearch.com/signs/videos/ability.mp4,0,384,180,46,579,400,able
3921,25,-1,1,12,12,aslsearch,train,http://www.aslsearch.com/signs/videos/accept.mp4,0,603,177,46,579,400,accept
567,25,-1,1,15,11,signingsavvy,test,https://www.signingsavvy.com/signs/mp4/7/7493.mp4,0,634,61,12,227,192,accident
555,25,-1,1,1,12,aslsearch,train,http://www.aslsearch.com/signs/videos/accident...,0,639,180,45,574,400,accident
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10279,25,-1,1,0,117,aslbrick,val,http://aslbricks.org/New/ASL-Videos/wrap.mp4,0,69542,385,33,1003,720,wrap
1331,25,-1,1,1,115,aslbrick,test,http://aslbricks.org/New/ASL-Videos/write.mp4,0,69544,375,42,940,720,write
1049,25,-1,1,1,118,aslbrick,test,http://aslbricks.org/New/ASL-Videos/yellow.mp4,0,69545,256,41,883,720,yellow
201,25,-1,1,1,118,aslbrick,val,http://aslbricks.org/New/ASL-Videos/yes.mp4,0,69546,214,41,881,720,yes


## Prepare dataset

In [5]:
encoder = pickle.load(open("one hot encoder", 'rb'))
y1 = encoder.transform(np.array(list(df1["gloss"])).reshape(-1,1)).toarray()
y2 = encoder.transform(np.array(list(df2["gloss"])).reshape(-1,1)).toarray()

In [6]:
le = pickle.load(open("label encoder", 'rb'))
y_1 = le.transform(np.array(list(df1["gloss"])))
y_2 = le.transform(np.array(list(df2["gloss"])))

In [7]:
y_1.shape

(2789,)

In [8]:
print(X1.shape, y1.shape)

(2789, 64, 64, 64, 3) (2789, 1868)


In [9]:
IMG_SHAPE = X1.shape[1:]
nb_classes = y1.shape[-1]

## Model implementation

In [10]:
mobileNet = tf.keras.applications.MobileNetV3Small(
    input_shape=IMG_SHAPE[1:],
    alpha=1.0,
    include_top=False,
    weights="imagenet"
)

In [15]:
input_layer = Input(shape=IMG_SHAPE)
split_layer = tf.split(input_layer, IMG_SHAPE[0], axis=1, num=None, name='split')
features = []
for layer in split_layer:
    reshape_layer = Reshape(IMG_SHAPE[1:])(layer)
    feature = mobileNet(reshape_layer)
    feature_flat = Flatten()(feature)
    features.append(feature_flat)
features = tf.stack(features, axis = 1)
lstm_layer = LSTM(64)(features)
dense_layer = Dense(128, activation='relu')(lstm_layer)
output_layer = Dense(nb_classes, activation='softmax')(dense_layer)

model = Model(input_layer, output_layer)
model.compile(loss='CategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 64, 64, 64,  0           []                               
                                 3)]                                                              
                                                                                                  
 tf.split_1 (TFOpLambda)        [(None, 1, 64, 64,   0           ['input_3[0][0]']                
                                3),                                                               
                                 (None, 1, 64, 64,                                                
                                3),                                                               
                                 (None, 1, 64, 64,                                          

In [17]:
model.fit(X1, y1, shuffle=True, epochs = 10, batch_size=64, validation_data=(X2, y2))

Epoch 1/10
44/44 [==============================] - 2859s 66s/step - loss: 7.5582 - accuracy: 0.0000e+00 - val_loss: 7.5316 - val_accuracy: 0.0000e+00
Epoch 2/10
44/44 [==============================] - 3914s 90s/step - loss: 7.5202 - accuracy: 0.0011 - val_loss: 7.5449 - val_accuracy: 0.0011
Epoch 3/10
44/44 [==============================] - 7408s 170s/step - loss: 7.3996 - accuracy: 0.0011 - val_loss: 7.9115 - val_accuracy: 0.0000e+00
Epoch 4/10
 8/44 [====>.........................] - ETA: 59:35 - loss: 7.1939 - accuracy: 0.0020  

In [ ]:
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.legend(["Training set loss", "Validation set loss", "Accuracy", "Validation accuracy"])

plt.yscale("log")
plt.show()

In [ ]:
y_pred1 = model.predict(X1)         
y_pred1 = np.argmax(y_pred1, axis = 1)

In [ ]:
print("F1 score : ", f1_score(y_1, y_pred1, average = 'weighted'))
print("Accuracy : ", accuracy_score(y_1, y_pred1))
print("Precision score : ", precision_score(y_1, y_pred1, average = 'weighted'))
print("Recall score : ", recall_score(y_1, y_pred1, average = 'weighted'))

In [ ]:
y_pred2 = model.predict(X2)         
y_pred2 = np.argmax(y_pred2, axis = 1)

In [ ]:
print("F1 score : ", f1_score(y_2, y_pred2, average = 'weighted'))
print("Accuracy : ", accuracy_score(y_2, y_pred2))
print("Precision score : ", precision_score(y_2, y_pred2, average = 'weighted'))
print("Recall score : ", recall_score(y_2, y_pred2, average = 'weighted'))

### Training only with hands landmarks

In [ ]:
X2 = X[:,:,:42]

In [ ]:
input_dim_2 = X2.shape[1:]

In [ ]:
model2 = Sequential()
model2.add(Input(shape=(input_dim_2)))
model2.add(Conv2D(filters=8, kernel_size= 3, strides = (1, 2), padding='same', activation='relu'))
model2.add(Conv2D(filters=16, kernel_size=(input_dim[0], 3), strides = (1, 2), padding='same', activation='relu'))
model2.add(Conv2D(filters=32, kernel_size= 3, strides = (1, 2),padding='same', activation='relu'))
model2.add(tf.keras.layers.Reshape((100, 6*32)))
model2.add(LSTM(64))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(nb_classes, activation='softmax'))
model2.compile(loss='CategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model2.summary()

In [ ]:
for train_index, test_index in sss.split(X2, y_):
    X_train, X_test = X2[train_index], X2[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model2.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))
    
    plt.plot(model2.history.history['loss'])
    plt.plot(model2.history.history['val_loss'])
    plt.plot(model2.history.history['accuracy'])
    plt.plot(model2.history.history['val_accuracy'])
    plt.legend(["Training set loss", "Validation set loss", "Accuracy", "Validation accuracy"])

    plt.yscale("log")
    plt.show()

In [ ]:
y_pred = model2.predict(X2)         
y_pred = np.argmax(y_pred, axis = 1)

In [ ]:
print("F1 score : ", f1_score(y_, y_pred, average = 'weighted'))
print("Accuracy : ", accuracy_score(y_, y_pred))
print("Precision score : ", precision_score(y_, y_pred, average = 'weighted'))
print("Recall score : ", recall_score(y_, y_pred, average = 'weighted'))

### Less complicated architecture

In [ ]:
model = Sequential()
model.add(Input(shape=(input_dim)))
model.add(tf.keras.layers.Reshape((100, 520*3)))
model.add(LSTM(128))
model.add(Dense(nb_classes, activation='softmax'))
model.compile(loss='CategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=1)

In [ ]:
for train_index, test_index in sss.split(X, y_):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))
    
    plt.plot(model.history.history['loss'])
    plt.plot(model.history.history['val_loss'])
    plt.plot(model.history.history['accuracy'])
    plt.plot(model.history.history['val_accuracy'])
    plt.legend(["Training set loss", "Validation set loss", "Accuracy", "Validation accuracy"])

    plt.yscale("log")
    plt.show()

In [ ]:
y_pred = model.predict(X)         
y_pred = np.argmax(y_pred, axis = 1)

In [ ]:
print("F1 score : ", f1_score(y_, y_pred, average = 'weighted'))
print("Accuracy : ", accuracy_score(y_, y_pred))
print("Precision score : ", precision_score(y_, y_pred, average = 'weighted'))
print("Recall score : ", recall_score(y_, y_pred, average = 'weighted'))